In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt
from pyod.models.lmdd import LMDD
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [11]:
!pip install pyod

You should consider upgrading via the '/home/sid2018-4/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
PROJECT_PATH = "/home/sid2018-4/Documents/M2 SID/pip2022/ProjectInterPromo/"
DATA_PATH = "dev/analyse_preliminaire/code/louis/Donnees_v0_NAN_supprimes.csv"


In [3]:
def evaluation(score: np.ndarray, y: np.ndarray, display=False):
    precision, recall, thresholds = precision_recall_curve(y, score)
    f1_score = 2 * precision * recall / (precision + recall)
    
    value = auc(recall, precision)
    
    if display:
        plt.plot(recall, precision)
        plt.show()
    
    return value, f1_score, precision, recall

In [4]:
# Lecture du fichier de donnée
data_v0 = pd.read_csv(PROJECT_PATH + DATA_PATH)

print("Nombre de fraude :", sum(data_v0["TOP_FRAUDE"]))
print("Nombre de données : ", len(data_v0))
print("Ratio : ", sum(data_v0["TOP_FRAUDE"]) / len(data_v0) * 100)

# Suppression des variables qualitative
quali_col = []
count = 0
for i, name in zip(data_v0.dtypes, data_v0.columns):
    if i == "object":
        quali_col.append(name)

data_quanti = data_v0.drop(columns=quali_col + ["Unnamed: 0"])

# 
data_quanti = data_quanti.sample(100000)

print("------------ Sur un echantillion ------------")
print("Nombre de fraude :", sum(data_quanti["TOP_FRAUDE"]))
print("Nombre de données : ", len(data_quanti))
print("Ratio : ", sum(data_quanti["TOP_FRAUDE"]) / len(data_quanti) * 100)

Nombre de fraude : 1479
Nombre de données :  644348
Ratio :  0.22953435100287423
------------ Sur un echantillion ------------
Nombre de fraude : 248
Nombre de données :  100000
Ratio :  0.248


In [5]:
train, test = train_test_split(data_quanti, test_size=0.3, random_state=43)
print(test['TOP_FRAUDE'].sum())

X = data_quanti.drop(columns=['TOP_FRAUDE'])
y = data_quanti['TOP_FRAUDE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=43)

81


In [ ]:
#Knn

In [48]:
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(X_train, y_train)

y_pred = neigh.predict(X_test)
test['knn_pred'] = y_pred

In [49]:
print(y_pred.sum())
print(y_test.sum())

0
59


In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9980333333333333

In [55]:
LMDDmodel = LMDD(contamination=0.1, n_iter=10, dis_measure='aad', random_state=None)

In [56]:
LMDDmodel.fit(train)

KeyboardInterrupt: 

In [6]:
#Isolation Forest
from sklearn.ensemble import IsolationForest

In [7]:
train = train.drop(columns=['TOP_FRAUDE', 'TOP_FRAUDE_CARTE', 'TOP_FRAUDE_VIREMENT'])

In [8]:
ytest = test['TOP_FRAUDE']
xtest = test.drop(columns=['TOP_FRAUDE', 'TOP_FRAUDE_CARTE', 'TOP_FRAUDE_VIREMENT'])

In [9]:
modelIForest = IsolationForest(random_state = 56)
modelIForest.fit(train)

IsolationForest(random_state=56)

In [10]:
results = modelIForest.predict(xtest)


In [11]:
print(results)

[1 1 1 ... 1 1 1]


In [12]:
count = 0
for i in range(len(results)):
    if results[i] == -1:
        count += 1
print(count)

194


In [13]:
score = [1 if x==-1 else 0 for x in results]

In [15]:
value, f1_score, precision, recall = evaluation(score, xtest, display=True)

ValueError: continuous-multioutput format is not supported